# This notebook is used to calculate the gamma + gamma -> pi0 matching efficiency & purity
### For the statistical uncertainties, the beta binomial distribution is used.

In [1]:
import sys
#sys.path.append("/home/rberner/cernbox/PhD/pi0_reconstruction/reco_software/lartpc_mlreco3d")   # locally
#sys.path.append("/home/rberner/cernbox/PhD/pi0_reconstruction/reco_software/pi0_reco")          # locally
#sys.path.append("/u/nu/rberner/pi0_reconstruction/lartpc_mlreco3d")                            # nu-gpu0x
#sys.path.append("/u/nu/rberner/pi0_reconstruction/pi0_reco")                                   # nu-gpu0x
sys.path.append("../../lartpc_mlreco3d")                                                          # JupyterLab
sys.path.append("../../pi0_reco")                                                                 # JupyterLab

In [3]:
io_cfg = '''
iotool:
  batch_size: 1
  shuffle: False
  num_workers: 4
  collate_fn: CollateSparse
  dataset:
    name: LArCVDataset
    data_keys:
      #- ../../data/test.root
      - ../../data/pi0_dunend_v1_p00.root
      #- ../../data/pi0_dunend_v2_p00.root
    limit_num_files: 1
    schema:
      input_data:
        - parse_sparse3d_scn
        - sparse3d_pcluster
      segment_label:
        - parse_sparse3d_scn
        - sparse3d_pcluster_semantics
      semantics:
        - parse_sparse3d_scn
        - sparse3d_pcluster_semantics
      dbscan_label:
        - parse_cluster3d_clean_full
        - cluster3d_pcluster
        - particle_corrected
        - sparse3d_pcluster_semantics
      particles_label:
        - parse_particle_points
        - sparse3d_pcluster
        - particle_corrected
      particles:
        - parse_particle_asis
        - particle_corrected
        - cluster3d_pcluster
      cluster_label:
        - parse_cluster3d_full
        - cluster3d_pcluster
        - particle_corrected
'''

# Convert the string to a dictionary
#import yaml
#cfg = yaml.load(cfg,Loader=yaml.Loader)

# Pre-process configuration
#from mlreco.main_funcs import process_config
#process_config(cfg)

In [4]:
from mlreco.main_funcs import prepare, apply_event_filter
from mlreco.utils.gnn.evaluation import node_assignment, node_assignment_bipartite, edge_assignment_score, clustering_metrics, node_assignment_score

In [5]:
# Configuration of the reconstruction chain
chain_cfg = '''
name: pi0_chain_fiducialized
#net_cfg: ../../config_files/network_configs/fullChain_gnn_dbscan_gpu0.cfg           # for the lartpc_mlreco3d/cnn_clustering branch
net_cfg: ../../config_files/network_configs/train_cluster_chain_gnn_dbscan_new.cfg   # for the lartpc_mlreco3d/full_chain branch
#net_cfg: ../../config_files/network_configs/fullChain_grappa_20210128.cfg
segment:                   uresnet                     # label, uresnet
deghost:                   label                       # label, uresnet
charge2energy:                                         # null, constant, average(, full, enet)
charge2energy_cst:         0.0082                      # energy response constant (0.0082 for mask, 0.0052 for uresnet)
charge2energy_average:     0.877                       # energy response average (1.3693 for mask, 0.877 for uresnet)
shower_start:              ppn                         # label, gnn, ppn (if using gnn -> use segment: uresnet)
shower_fragment:           gnn                         # label, dbscan, gnn
shower_direction:          pca                         # label, pca, cent
shower_cluster:            gnn                         # label, cone, gnn
shower_cluster_params:
  IP: 40
  Distance: 300
shower_energy:             pixel_sum                   # label, pixel_sum
shower_match:              ppn                         # label, proximity, ppn
refit_dir:                 true                        # true, false
refit_cone:                true                        # true, false
fiducialize:               14                          # Number of pixels to be removed from edge of LAr volume (pixel pitch: 3mm)
PPN_score_thresh:          [0.5, 0.5, 0.5, 0.5, 0.5]   # For PPN predictions to be classified as shower, track, michel, delta or LEScatter, respectively
analyse_true:              True                        # Whether the analyser module for (simulated) true data is used or not
analyse_reco:              True                        # Whether the analyser module for (simulated or real) reco data is used or not
'''

import yaml
chain_cfg = yaml.load(chain_cfg,Loader=yaml.Loader)

In [6]:
# Progress bar
from IPython.display import HTML, display
def progress(count, total, unit, message=''):
    return HTML("""
        <progress 
            value='{count}'
            max='{total}',
            style='width: 30%'
        >
            {count}
        </progress> {count}/{total} {unit} ({frac}%) ... {message}
    """.format(count=count, total=total, unit=unit, frac=int(float(count)/float(total)*100.),message=message))

In [7]:
class Results(object):
    '''
    Class for the storage of result acquired during chain.run_loop() process.
    The attributes are added in a dynamic way.
    This class can be used as:
    res = Results()
    res.attribute = someobject
    Use Results.__dict__ to show attributes.
    '''

    # Defined attributes
    # -------------------------------------
    '''
    See below...
    '''
    
    pass

In [8]:
# Some imports
import numpy as np
from larcv import larcv
from numpy import linalg

Welcome to JupyROOT 6.16/00


In [9]:
# Initialize the chain
from pi0.chain import Pi0Chain
chain = Pi0Chain(io_cfg, chain_cfg)

Initialized Pi0 mass chain, log path: masses_fiducialized_14px.csv
 3:  {'iotool': {'batch_size': 1, 'shuffle': False, 'num_workers': 8, 'collate_fn': 'CollateSparse', 'sampler': {'name': 'RandomSequenceSampler'}, 'dataset': {'name': 'LArCVDataset', 'data_keys': ['/home/frans/slac/dlp/data/cluster_new/train.root'], 'limit_num_files': 10, 'schema': {'input_data': ['parse_sparse3d_scn', 'sparse3d_pcluster'], 'segment_label': ['parse_sparse3d_scn', 'sparse3d_pcluster_semantics'], 'cluster_label': ['parse_cluster3d_clean_full', 'cluster3d_pcluster', 'particle_pcluster', 'sparse3d_pcluster_semantics'], 'particles_label': ['parse_particle_points', 'sparse3d_pcluster', 'particle_corrected']}}}, 'model': {'name': 'full_chain', 'modules': {'chain': {'verbose': True, 'enable_uresnet': True, 'enable_ppn': True, 'enable_dbscan': True, 'enable_cnn_clust': False, 'enable_gnn_shower': True, 'enable_gnn_inter': True, 'enable_ghost': False, 'use_ppn_in_gnn': True}, 'full_chain_loss': {'segmentation_wei

KeyError: 'shower_direction'

In [ ]:
# Define a list with the results obtained by run_loop()
ResultsList = []

# Loop over dataset
data_size  = 2000 #len(chain.hs.data_io)

out = display(progress(0,data_size,'images'),display_id=True)
for event in range(data_size):
    chain.run_loop()
    assert chain.true_info['ev_id']            == chain.event['index']

    # Event selection:
    # ----------------
    skip_event = False
    skip_event_list = []
    # Continue if no true pi0 is in the event
    if chain.true_info['n_pi0'] == 0:
        #print(' === FAIL: n_pi0     =', chain.true_info['n_pi0'])
        skip_event = True
        skip_event_list.append(0)
        #out.update(progress(event,data_size,'images'))
        #continue
    
    # Select events with exactly 1 true pi0 -> gamma+gamma
    if chain.true_info['n_gammas'] != 2:
        #print(' === FAIL: n_gammas  =', chain.true_info['n_gammas'])
        skip_event = True
        skip_event_list.append(1)
        #out.update(progress(event,data_size,'images'))
        #continue
    #if len(chain.output['showers']) != 2:
    #    out.update(progress(event,data_size,'images'))
    #    skip_event = True
    #    continue
    
    # Select events where each photon of a pi0 decay has at least 20 MeV energy deposition
    for true_gamma_0 in range(chain.true_info['n_pi0']):
        if 2*chain.true_info['n_pi0'] != chain.true_info['n_gammas']:
            print('WARNING: true n_pi0s =', chain.true_info['n_pi0'], 'does not correspond to true n_gammas =', chain.true_info['n_gammas'])
        else:
            if chain.true_info['gamma_edep'][2*true_gamma_0]<20. or chain.true_info['gamma_edep'][2*true_gamma_0+1]<20.:
                skip_event = True
                skip_event_list.append(2)
    
    # Continue if true gamma ekin/edep < 0.8:
    for true_gamma_0 in range(chain.true_info['n_pi0']):
        if 2*chain.true_info['n_pi0'] != chain.true_info['n_gammas']:
            print('WARNING: true n_pi0s =', chain.true_info['n_pi0'], 'does not correspond to true n_gammas =', chain.true_info['n_gammas'])
        else:
            if chain.true_info['gamma_edep'][2*true_gamma_0] > 0. and chain.true_info['gamma_edep'][2*true_gamma_0+1] > 0.:
                if chain.true_info['gamma_ekin'][2*true_gamma_0]/chain.true_info['gamma_edep'][2*true_gamma_0] < 0.8 or\
                   chain.true_info['gamma_ekin'][2*true_gamma_0+1]/chain.true_info['gamma_edep'][2*true_gamma_0+1] < 0.8:
                    #print(' === FAIL: true ekin/edep =', chain.true_info['gamma_ekin']/chain.true_info['gamma_edep'], '< 0.8')
                    skip_event = True
                    skip_event_list.append(3)
                    #out.update(progress(event,data_size,'images'))
                    #continue
    
    # Continue if >= 1 true gamma is OOFV:
    #if len(chain.true_info['OOFV'])>0:
        #print(' === FAIL: true OOFV =', chain.true_info['OOFV'])
        #skip_event = True
        #skip_event_list.append(4)
        #out.update(progress(event,data_size,'images'))
        #continue
    
    if skip_event:
        out.update(progress(event,data_size,'images'))
        skip_event = False
        print(' Skip event', chain.event['index'], '(failed selection cuts:', skip_event_list, ')')
        continue
    
    # Extract results:
    # ----------------
    # Instantiate Results class
    extracted_data = Results()
    
    # Add members of the Results class
    extracted_data.event_id                    = chain.true_info['ev_id']

    extracted_data.true_n_pi0s                 = chain.true_info['n_pi0']
    extracted_data.true_n_gammas               = chain.true_info['n_gammas']
    extracted_data.true_pi0_track_ids          = chain.true_info['pi0_track_ids']
    extracted_data.true_gamma_group_ids        = chain.true_info['gamma_group_ids']
    extracted_data.true_shower_particle_ids    = chain.true_info['shower_particle_ids']
    extracted_data.true_gamma_ids_making_compton_scat = chain.true_info['gamma_ids_making_compton_scat']
    extracted_data.true_pi0_ekin               = chain.true_info['pi0_ekin']
    extracted_data.true_gamma_pos              = chain.true_info['gamma_pos']
    extracted_data.true_gamma_dir              = chain.true_info['gamma_dir']
    extracted_data.true_gamma_mom              = chain.true_info['gamma_mom']
    extracted_data.true_gamma_ekin             = chain.true_info['gamma_ekin']
    extracted_data.true_gamma_edep             = chain.true_info['gamma_edep']
    extracted_data.true_gamma_voxels           = chain.true_info['gamma_voxels']
    extracted_data.true_gamma_n_voxels         = chain.true_info['gamma_n_voxels']
    extracted_data.true_gamma_first_step       = chain.true_info['gamma_first_step']
    extracted_data.compton_electron_first_step = chain.true_info['compton_electron_first_step']
    extracted_data.true_shower_first_edep      = chain.true_info['shower_first_edep']
    extracted_data.true_OOFV                   = chain.true_info['OOFV']
    extracted_data.true_gamma_angle            = chain.true_info['gamma_angle'] # [rad]
    extracted_data.true_pi0_mass               = chain.true_info['pi0_mass']
    
    extracted_data.reco_n_pi0s                 = chain.reco_info['n_pi0']
    extracted_data.reco_n_gammas               = chain.reco_info['n_gammas']
    extracted_data.reco_matches                = chain.reco_info['matches']
    extracted_data.reco_gamma_mom              = chain.reco_info['gamma_mom']
    extracted_data.reco_gamma_dir              = chain.reco_info['gamma_dir']
    extracted_data.reco_gamma_start            = chain.reco_info['gamma_start']
    extracted_data.reco_gamma_edep             = chain.reco_info['gamma_edep']
    extracted_data.reco_gamma_pid              = chain.reco_info['gamma_pid']
    extracted_data.reco_gamma_voxels_mask      = chain.reco_info['gamma_voxels_mask']
    extracted_data.reco_gamma_n_voxels_mask    = chain.reco_info['gamma_n_voxels_mask']
    extracted_data.reco_gamma_voxels           = chain.reco_info['gamma_voxels']
    extracted_data.reco_gamma_n_voxels         = chain.reco_info['gamma_n_voxels']
    extracted_data.reco_OOFV                   = chain.reco_info['OOFV']
    extracted_data.reco_gamma_angle            = chain.reco_info['gamma_angle'] # [rad]
    extracted_data.reco_pi0_mass               = chain.reco_info['pi0_mass']
    
    #for sh in range(extracted_data.true_n_gammas):
        #print(' true start: ', extracted_data.true_gamma_first_step[sh])
        #print(' true dir: ', extracted_data.true_gamma_dir[sh])
        #print(' ------------------------------------------------ ')

    extracted_data.clusters_n_clusters = len(chain.output['showers'])
    extracted_data.clusters_start      = []
    extracted_data.clusters_dir        = []
    extracted_data.clusters_voxels     = []
    extracted_data.clusters_energy     = []
    for i, shower in enumerate(chain.output['showers']):
        extracted_data.clusters_start.append(shower.start)
        extracted_data.clusters_dir.append(shower.direction)
        extracted_data.clusters_voxels.append(shower.voxels)
        extracted_data.clusters_energy.append(shower.energy)


    # Append the extracted data to the ResultsList
    ResultsList.append(extracted_data)
    
    
    # Print to screen
    # ----------------
    print_information = False #(extracted_data.true_n_pi0s+extracted_data.true_n_pi0s)>0 #True #(len(extracted_data.reco_pi0_mass)>0)
    if print_information: # and (extracted_data.true_n_pi0s>0 or extracted_data.reco_n_pi0s>0):
        print(' event_id:                    ', extracted_data.event_id)
        print(' true_n_pi0s:                 ', extracted_data.true_n_pi0s)
        print(' reco_n_pi0s:                 ', extracted_data.reco_n_pi0s)
        print(' true_n_gammas:               ', extracted_data.true_n_gammas)
        print(' reco_n_gammas:               ', extracted_data.reco_n_gammas)
        #print(' clusters_n_clusters:         ', extracted_data.clusters_n_clusters)
        #print(' true_pi0_track_ids:          ', extracted_data.true_pi0_track_ids)
        #print(' true_gamma_group_ids:        ', extracted_data.true_gamma_group_ids)
        #print(' true_shower_particle_ids:    ', extracted_data.true_shower_particle_ids)
        #print(' true_gamma_ids_making_compt: ', extracted_data.true_gamma_ids_making_compton_scat)
        print(' true_pi0_ekin:               ', extracted_data.true_pi0_ekin)
        print(' true pi0 vertex:             ', extracted_data.true_gamma_pos)
        print(' true_gamma_pos:              ', extracted_data.true_gamma_pos)
        print(' true_gamma_dir:              ', extracted_data.true_gamma_dir)
        print(' reco_gamma_dir:              ', extracted_data.reco_gamma_dir)
        #print(' clusters_dir:                ', extracted_data.clusters_dir)
        print(' true_gamma_mom:              ', extracted_data.true_gamma_mom)
        print(' reco_gamma_mom:              ', extracted_data.reco_gamma_mom)
        print(' true_gamma_ekin:             ', extracted_data.true_gamma_ekin)
        print(' true_gamma_edep:             ', extracted_data.true_gamma_edep)
        print(' reco_gamma_edep:             ', extracted_data.reco_gamma_edep)
        #print(' true_gamma_voxels:           ', extracted_data.true_gamma_voxels)
        print(' true_gamma_n_voxels:         ', extracted_data.true_gamma_n_voxels)
        print(' reco_gamma_n_voxels:         ', extracted_data.reco_gamma_n_voxels)
        print(' true_OOFV:                   ', extracted_data.true_OOFV)
        print(' reco_OOFV:                   ', extracted_data.reco_OOFV)
        print(' true_gamma_first_step:       ', extracted_data.true_gamma_first_step)
        print(' compton_electron_first_step: ', extracted_data.compton_electron_first_step)
        print(' true_shower_first_edep:      ', extracted_data.true_shower_first_edep)
        print(' reco_gamma_start:            ', extracted_data.reco_gamma_start)
        #print(' clusters_start:              ', extracted_data.clusters_start)
        #print(' cluster_start: ')
        #for index in range(extracted_data.clusters_n_clusters):
        #    print('                  ', extracted_data.clusters_start[index])
        print(' matches:                     ', extracted_data.reco_matches)
        print(' true_gamma_angle [rad]:      ', extracted_data.true_gamma_angle)
        print(' reco_gamma_angle [rad]:      ', extracted_data.reco_gamma_angle)
        print(' true_pi0_mass:               ', extracted_data.true_pi0_mass)
        print(' reco_pi0_mass:               ', extracted_data.reco_pi0_mass)
        #print(' ------------------------------------------------------ ')
    
    print(' === Event', extracted_data.event_id, 'passed selection. N true/reco pi0:', extracted_data.true_n_pi0s, '/', extracted_data.reco_n_pi0s)
    
    ##for clust in range(extracted_data.clusters_n_clusters):
    ##    print(' clust: ', clust)
    ##    print(len(chain.output['forward']['points'][0][clust][:]))
    
    
    # Draw event:
    # ----------------
    # Draw the last event if requested
    #i_draw    = 0
    #n_draw    = 5
    #if 'matches' in chain.output and len(chain.output['matches']): # and i_draw < n_draw:
    #    chain.draw()
    #    i_draw += 1

    #draw_eventIDs = [] #[2,4,5,6] #[8,14,15,16] #[1,4,5,7,8,53,54,68,70,71,73,76,78,79,80,83,84]
    #if chain.true_info['ev_id'] in draw_eventIDs:
    #    chain.draw()
    
    #if chain.true_info['n_gammas']==2 and chain.reco_info['n_gammas']==2:
    #    chain.draw()

    #if chain.reco_info['n_pi0']>0:
    #    chain.draw()
    
    #if chain.reco_info['n_pi0']>0:
    #    chain.draw()
    
    #if chain.true_info['n_pi0'] != chain.reco_info['n_pi0']:
    #    chain.draw()
    
    #chain.draw()
    
    #if print_information:
    #    print(' ------------------------------------------------------------------------------------------- ')

    # Update progress bar
    out.update(progress(event,data_size,'images'))
out.update(progress(data_size,data_size,'images'))

print('Done.')

# Efficiency and Purity
### Note: Classify photon as properly matched if their
- reconstructed start position is closer than "spatial_tolerance" (a few px) from the true shower's start (= true photon's first step) and
- reconstructed direction is in agreement (up to "angular_tolerance") with the true shower's direction
- TODO: Could also consider voxel overlap between reco shower and true shower

In [ ]:
class matching_eff_pur(object):
    '''
    Class for the storage of pi0 matching efficiency and purity
    '''
    # Defined attributes
    # -------------------------------------
    '''
    event_id
    true_n_pi0s
    reco_n_pi0s
    true_gamma_edep
    reco_gamma_edep
    true_pi0_ekin
    true_gamma_mom
    correctly_matched_pi0_ids
    wrongly_matched_pi0_ids
    correctly_matched_n_pi0
    wrongly_matched_pi0_ids
    correctly_matched_pi0_ids
    '''
    
    pass

In [ ]:
eff_pur_list = []
print_info   = False

# Progress bar
out = display(progress(0,len(ResultsList),'images'),display_id=True)

for event, res in enumerate(ResultsList):
    
    if print_info:
        print(' ================================================================ ')
        print(' event:               ', i)
        print(' event ID:            ', res.event_id)
        print(' true n pi0:          ', res.true_n_pi0s)
        print(' reco n pi0:          ', res.reco_n_pi0s)
        print(' true n gammas:       ', res.true_n_gammas)
        print(' reco n gammas:       ', res.reco_n_gammas)
        print(' true pi0 ekin:       ', res.true_pi0_ekin)
        print(' true gamma pos:      ', res.true_gamma_pos)
        print(' true gamma 1st step: ', res.true_gamma_first_step)
        print(' true gamma 1st edep: ', res.true_shower_first_edep)
        print(' reco gamma start:    ', res.reco_gamma_start)
        print(' true gamma dir:      ', res.true_gamma_dir)
        print(' reco gamma dir:      ', res.reco_gamma_dir)
        print(' true gamma mom:      ', res.true_gamma_mom)
        print(' reco gamma mom:      ', res.reco_gamma_mom)
        print(' true gamma ekin:     ', res.true_gamma_ekin)
        print(' true gamma edep:     ', res.true_gamma_edep)
        print(' reco gamma edep:     ', res.reco_gamma_edep)
        #print(' true_gamma_voxels mask:   ', res.true_gamma_voxels_mask)
        #print(' reco gamma voxels mask:   ', res.reco_gamma_voxels_mask)
        #print(' true_gamma n voxels mask: ', res.true_gamma_n_voxels_mask)
        #print(' reco gamma n voxels mask: ', res.reco_gamma_n_voxels_mask)
        #print(' true_gamma_voxels:        ', res.true_gamma_voxels)
        #print(' reco gamma voxels:        ', res.reco_gamma_voxels)
        #print(' reco gamma pid:           ', res.reco_gamma_pid)
        print(' true gamma n voxels: ', res.true_gamma_n_voxels)
        print(' reco gamma n voxels: ', res.reco_gamma_n_voxels)
        print(' true OOFV:           ', res.true_OOFV)
        print(' reco OOFV:           ', res.reco_OOFV)
        print(' true gamma angle:    ', res.true_gamma_angle)
        print(' reco gamma angle:    ', res.reco_gamma_angle)
        print(' true pi0 mass:       ', res.true_pi0_mass)
        print(' reco pi0 mass:       ', res.reco_pi0_mass)
        print(' reco matches:        ', res.reco_matches)

        #print(' clusters n clusters: ', res.clusters_n_clusters)
        print(' clusters start:      ', res.clusters_start)
        print(' clusters dir:        ', res.clusters_dir)
        #print(' clusters voxels:     ', res.clusters_voxels)
        print(' clusters energy:     ', res.clusters_energy)
    
    
    # Instantiate matching_eff_pur class
    eff_pur_data = matching_eff_pur()
    
    eff_pur_data.event_id                  = res.event_id
    eff_pur_data.true_n_pi0s               = res.true_n_pi0s
    eff_pur_data.reco_n_pi0s               = res.reco_n_pi0s
    eff_pur_data.true_gamma_edep           = res.true_gamma_edep
    eff_pur_data.reco_gamma_edep           = res.reco_gamma_edep
    eff_pur_data.correctly_matched_pi0_ids = []
    eff_pur_data.wrongly_matched_pi0_ids   = []
    eff_pur_data.true_pi0_ekin             = []
    eff_pur_data.true_gamma_mom            = []
    for k in range(res.true_n_pi0s):
        eff_pur_data.true_pi0_ekin.append(res.true_pi0_ekin[k])
        eff_pur_data.true_gamma_mom.append(res.true_gamma_mom[2*k])
        eff_pur_data.true_gamma_mom.append(res.true_gamma_mom[2*k+1])

    # Define the tolerances for a reco shower to be accepted as properly matched to a true shower
    spatial_tolerance = 20 # [px]
    angular_tolerance = 20 # [degrees]
    
    # Loop over all matched photons:
    if res.reco_n_gammas != 0:
        # Build a separation matrix and angle matrix
        sep_mat = np.full((int(res.true_n_gammas), int(res.reco_n_gammas)), float('inf')) # true_gammas vertically, reco_gammas horizontally
        ang_mat = np.full((int(res.true_n_gammas), int(res.reco_n_gammas)), float('inf')) # true_gammas vertically, reco_gammas horizontally
        
        for i in range(int(res.true_n_gammas)):
            for j in range(int(res.reco_n_gammas)):
                #print(' i: ', i, ' \t j: ', j, ' \t ', (res.reco_gamma_start[j]-res.true_gamma_pos[i]))
                #print(' res.true_gamma_first_step', np.array(res.true_gamma_first_step[i]))
                #print(' res.reco_gamma_start', np.array(res.reco_gamma_start[j]))
                #distance = np.linalg.norm(np.array(res.true_gamma_first_step[i])-np.array(res.reco_gamma_start[j]))
                distance = np.linalg.norm(np.array(res.true_shower_first_edep[i])-np.array(res.reco_gamma_start[j]))
                angle    = np.arccos(np.dot(res.reco_gamma_dir[j],res.true_gamma_dir[i]))*360./(2.*np.pi)
                #print(' distance [px]: ', distance)
                #print(' angle [deg]:   ', angle)
                #if distance < spatial_tolerance:
                #    sep_mat[i,j] = distance
                sep_mat[i,j] = distance
                ang_mat[i,j] = angle
                #else:
                #    sep_mat[i,j] = float('inf')
                #if angle < angular_tolerance:
                #    ang_mat[i,j] = angle
                #else:
                #    ang_mat[i,j] = float('inf')
        #print(' true_gamma_first_step: ', res.true_gamma_first_step)
        #print(' true_shower_first_edep: ', res.true_shower_first_edep)
        #print(' reco_gamma_start: ', res.reco_gamma_start)
        #print(' sep_mat: \n', sep_mat)
        #print(' ang_mat: \n', ang_mat)
        
        for i in range(int(res.true_n_gammas)):
            for j in range(int(res.reco_n_gammas)):
                if (sep_mat[i][j] > spatial_tolerance) or (ang_mat[i][j] > angular_tolerance):
                    sep_mat[i][j] = float('inf')
                    ang_mat[i][j] = float('inf')
        #print(' sep_mat new: \n', sep_mat)
        #print(' ang_mat new: \n', ang_mat)
        
        
        # Assign reco_gamma to true_gamma (taking sequencially the smallest index from the sep_mat)
        assignments = [] # list of U arrays (where U is min(n_clusters,n_true_gammas)). First array entry: cluster index, second array: true_gamma index.
        for i in range(min(sep_mat.shape)):
            if sep_mat.min() < spatial_tolerance:
                indices = np.argwhere(sep_mat.min() == sep_mat)
                assignments.append(indices[0])
                sep_mat[indices[0][0],:] = float('inf')
                sep_mat[:,indices[0][1]] = float('inf')
            #else:
                #print(' sep_mat.min() =', sep_mat.min(), '>', spatial_tolerance, '= spatial_tolerance -> No action')
        #print(' assignments: ', assignments)
        #print(' sep_mat:\n', sep_mat)
        #print(' ang_mat:\n', ang_mat)
        #print(' len(assignments): ', len(assignments))
        #if len(assignments) < 2:
            #print(' eff_pur_data.correctly_matched_n_pi0 should be set to 0 ')
        #if len(assignments) > 2:
            #print(' WARNING: len(assignments) =', len(assignments), ' (assignments =', assignments, '[event: ', res.event_id, ']')


        # Check if the assignments should be rejected by the angle between true_dir and reco_dir
        reject_due_to_angle = []
        for i, ass in enumerate(assignments):
            if ang_mat[ass[0]][ass[1]] > angular_tolerance:
                reject_due_to_angle.append(i)

        # Check if assignments correspond (pair-wise) to the correct true_pi0
        # if assignment = (x,y) = (even-even), then (x+1, y+1) should be found in assignments in order to be properly matched
        # if assignment = (x,y) = (even-odd),  then (x+1, y-1) should be found in assignments in order to be properly matched
        # if assignment = (x,y) = (odd-even),  then (x-1, y+1) should be found in assignments in order to be properly matched
        # if assignment = (x,y) = (odd-odd),   then (x-1, y-1) should be found in assignments in order to be properly matched
        counter_correctly_matched_pi0 = 0
        correctly_matched_pi0_indices = []
        counter_wrongly_matched_pi0   = 0
        wrongly_matched_pi0_indices   = []
        for i, ass in enumerate(assignments):
            proper_match = False
            #print(' ass: ', ass)
            if ass[0]%2 == 0 and ass[1]%2 == 0:
                #print(' even-even')
                for j, ass_test in enumerate(assignments):
                    #print(' ass_test: ', ass_test)
                    if (ass_test[0]==ass[0]+1) and (ass_test[1]==ass[1]+1):
                        #print(' proper match: ', ass_test)
                        proper_match = True
                        if ass[0]%2==0 and ass[0] not in correctly_matched_pi0_indices:
                            if i not in reject_due_to_angle and j not in reject_due_to_angle:
                                correctly_matched_pi0_indices.append(int(ass[0]/2))
                            else:
                                _ang = np.arccos(np.dot(res.reco_gamma_dir[ass[1]],res.true_gamma_dir[ass[0]]))*360./(2.*np.pi)
                                print(' Rejected assignment', ass, 'due to angle (',_ang, 'degrees)...')
                        break
                    #else:
                    #    print(' no match...')
            elif ass[0]%2 == 0 and ass[1]%2 == 1:
                #print(' even-odd')
                for j, ass_test in enumerate(assignments):
                    #print(' ass_test: ', ass_test)
                    if (ass_test[0]==ass[0]+1) and (ass_test[1]==ass[1]-1):
                        #print(' proper match: ', ass_test)
                        proper_match = True
                        if ass[0]%2==0 and ass[0] not in correctly_matched_pi0_indices:
                            if i not in reject_due_to_angle and j not in reject_due_to_angle:
                                correctly_matched_pi0_indices.append(int(ass[0]/2))
                            else:
                                _ang = np.arccos(np.dot(res.reco_gamma_dir[ass[1]],res.true_gamma_dir[ass[0]]))*360./(2.*np.pi)
                                print(' Rejected assignment', ass, 'due to angle (',_ang, 'degrees)...')
                        break
                    #else:
                    #    print(' no match...')
            elif ass[0]%2 == 1 and ass[1]%2 == 0:
                #print(' odd-even')
                for j, ass_test in enumerate(assignments):
                    #print(' ass_test: ', ass_test)
                    if (ass_test[0]==ass[0]-1) and (ass_test[1]==ass[1]+1):
                        #print(' proper match: ', ass_test)
                        proper_match = True
                        if ass[0]%2==0 and ass[0] not in correctly_matched_pi0_indices:
                            if i not in reject_due_to_angle and j not in reject_due_to_angle:
                                correctly_matched_pi0_indices.append(int(ass[0]/2))
                            else:
                                _ang = np.arccos(np.dot(res.reco_gamma_dir[ass[1]],res.true_gamma_dir[ass[0]]))*360./(2.*np.pi)
                                print(' Rejected assignment', ass, 'due to angle (',_ang, 'degrees)...')
                        break
                    #else:
                    #    print(' no match...')
            elif ass[0]%2 == 1 and ass[1]%2 == 1:
                #print(' odd-odd')
                for j, ass_test in enumerate(assignments):
                    #print(' ass_test: ', ass_test)
                    if (ass_test[0]==ass[0]-1) and (ass_test[1]==ass[1]-1):
                        #print(' proper match: ', ass_test)
                        proper_match = True
                        if ass[0]%2==0 and ass[0] not in correctly_matched_pi0_indices:
                            if i not in reject_due_to_angle and j not in reject_due_to_angle:
                                correctly_matched_pi0_indices.append(int(ass[0]/2))
                            else:
                                _ang = np.arccos(np.dot(res.reco_gamma_dir[ass[1]],res.true_gamma_dir[ass[0]]))*360./(2.*np.pi)
                                print(' Rejected assignment', ass, 'due to angle (',_ang, 'degrees)...')
                        break
            else:
                print(' WARNING: assignment', ass, 'not recognized!! [event: ', res.event_id, ']')
            
            if proper_match:
                counter_correctly_matched_pi0 += 1
                #print(' proper_match: ', proper_match)
            #if not proper_match:
                #print(' No proper match found ')
        #print(' correctly_matched_pi0_indices: ', correctly_matched_pi0_indices)
        for index in range(res.reco_n_pi0s):
            if index not in correctly_matched_pi0_indices:
                wrongly_matched_pi0_indices.append(index)
        #print(' wrongly_matched_pi0_indices:   ', wrongly_matched_pi0_indices)
        #print(' total matched:                 ', res.reco_n_pi0s)

        if (counter_correctly_matched_pi0%2 != 0):
            print(' ERROR: counter for correctly matched pairs must be an even number, but it is ', counter_correctly_matched_pi0, ' [event: ', res.event_id, ']')
            #assert (counter_correctly_matched_pi0%2 == 0)
        else:
            eff_pur_data.correctly_matched_n_pi0 = int(counter_correctly_matched_pi0/2)
        
        if len(correctly_matched_pi0_indices) > 0:
            eff_pur_data.correctly_matched_pi0_ids = correctly_matched_pi0_indices# = np.array(correctly_matched_pi0_indices)
        if len(wrongly_matched_pi0_indices) > 0:
            eff_pur_data.wrongly_matched_pi0_ids = wrongly_matched_pi0_indices# = np.array(wrongly_matched_pi0_indices)
        #else:
        #    eff_pur_data.correctly_matched_pi0_ids = []
        #print(eff_pur_data.correctly_matched_pi0_ids)
        #print(' ---------------------------------- ')

    #print(' -------------------------------------------------------------------------------------------------- ')


    # Append the extracted data to the ResultsList
    #print(eff_pur_data.event_id, eff_pur_data.true_n_pi0s, eff_pur_data.correctly_matched_n_pi0, eff_pur_data.reco_n_pi0s)
    eff_pur_list.append(eff_pur_data)
    
    # Update progress bar
    out.update(progress(res.event_id,len(ResultsList),'images'))
out.update(progress(len(ResultsList),len(ResultsList),'images'))
    
print(' Done ')

In [ ]:
'''
eff_pur_list = []

# Progress bar
out = display(progress(0,len(ResultsList),'images'),display_id=True)

for i, res in enumerate(ResultsList):
    
    #if i != 78:
    #    continue
    
    # Event Selection
    # =================================
    print_selection_info = False
    
    # Initialise boolean for event rejection
    reject_event = False
    
    # If true_n_pi0s is not exactly == 1: reject
    # --------------------------------------------------------------
    #if res.true_n_pi0s != 1:
    #    if print_selection_info:
    #        print(' WARNING: res.true_n_pi0s == ', res.true_n_pi0s, ' [event: ', res.event_id, ']')
    #    reject_event = True
    
    # If true_n_pi0s is == 0: reject
    # --------------------------------------------------------------
    #if res.true_n_pi0s == 0:
        #if print_selection_info:
            #print(' WARNING: res.true_n_pi0s == ', res.true_n_pi0s, ' [event: ', res.event_id, ']')
    #    reject_event = True
    
    # If no true and no reco pi0 is available: reject
    # --------------------------------------------------------------
    #if res.true_n_pi0s == 0 and res.reco_n_pi0s == 0:
    #    if print_selection_info:
    #        print(' WARNING: res.true_n_pi0s == ', res.true_n_pi0s , ' == res.reco_n_pi0s == ', res.res_n_pi0s, ' [event: ', res.event_id, ']')
    #    reject_event = True
    
    # If number of true photons != 2 * true_n_pi0s: reject
    # --------------------------------------------------------------
    if 2*res.true_n_pi0s != res.true_n_gammas:
        if print_selection_info:
            print(' WARNING: number of true pi0 (', res.true_n_pi0s, ') is not in agreement with number of true photons (', res.true_n_gammas, ') [event: ', res.event_id, ']')
            print(' IS ONE OF THE GAMMA LEAVING THE DETECTOR WITHOUT HAVING ANY EDEP? ')
        reject_event = True
    
    # If not both gammas of a true pi0 start in fiducial volume: reject
    # --------------------------------------------------------------
    if not reject_event:
        for true_pi0 in range(res.true_n_pi0s):
            if (np.any(np.array(res.true_gamma_first_step[true_pi0])<(0+chain.cfg['fiducialize'])) or\
                np.any(np.array(res.true_gamma_first_step[true_pi0])>(767-chain.cfg['fiducialize'])) or\
                np.any(np.array(res.true_gamma_first_step[true_pi0+1])<(0+chain.cfg['fiducialize'])) or\
                np.any(np.array(res.true_gamma_first_step[true_pi0+1])>(767-chain.cfg['fiducialize'])) or\
                np.any(np.array(res.true_shower_first_edep[true_pi0])<(0+chain.cfg['fiducialize'])) or\
                np.any(np.array(res.true_shower_first_edep[true_pi0])>(767-chain.cfg['fiducialize'])) or\
                np.any(np.array(res.true_shower_first_edep[true_pi0+1])<(0+chain.cfg['fiducialize'])) or\
                np.any(np.array(res.true_shower_first_edep[true_pi0+1])>(767-chain.cfg['fiducialize'])) ):
                if print_selection_info:
                    print(' WARNING: true shower 1st step or 1st edep is OOFV [event: ', res.event_id, ']')
                    print(' --- 1st step: ', res.true_gamma_first_step)
                    print(' --- 1st edep: ', res.true_shower_first_edep)
                reject_event = True
    #for true_pi0 in range(res.true_n_pi0s):
    #    if ( np.any(np.array(res.true_gamma_first_step[true_pi0])<0) or np.any(np.array(res.true_gamma_first_step[true_pi0])>767) or \
    #        np.any(np.array(res.true_gamma_first_step[true_pi0+1])<0) or np.any(np.array(res.true_gamma_first_step[true_pi0+1])>767)):
    #        if print_selection_info:
    #            print(' WARNING: true gamma 1st step is OOFV: ', res.true_gamma_first_step , ' [event: ', res.event_id, ']')
    #        reject_event = True
    #for true_gamma in range(res.true_n_gammas):
    #    if ( np.any(np.array(res.true_gamma_first_step[true_gamma])<0) or np.any(np.array(res.true_gamma_first_step[true_gamma])>767) ):
    #        if print_selection_info:
    #            print(' WARNING: true gamma 1st step is OOFV: ', res.true_gamma_first_step , ' [event: ', res.event_id, ']')
    #        reject_event = True
    
    # If not all true gammas are completely in FV: reject
    # --------------------------------------------------------------
    # Note: This cut removes quite a large number of events since many true photons do have 1 or more edep OOFV
    if len(res.true_OOFV)>0:
        if print_selection_info:
            print(' WARNING: > 0 true gamma has > 0 edep OOFV: ', res.true_OOFV , ' [event: ', res.event_id, ']')
        reject_event = True
    
    # If not all reco showers are in FV: reject
    # --------------------------------------------------------------
    if res.reco_OOFV:
        if print_selection_info:
            print(' WARNING: event_id: ', res.event_id, ' has reco_OOFV: ', res.reco_OOFV)
        reject_event = True
    
    # Proceed to next event if the current is rejected
    # --------------------------------------------------------------
    #if reject_event:
        #if print_selection_info:
    #    print(' Event has been rejected... ')
    #    print(' -------------------------------------------------------------------------------------------------- ')
        #continue
    
    
    # True pi0 Selection
    # =================================
    reject_pi0 = [False for _ in range(res.true_n_pi0s)]
    
    # If at least one true photon is not making an interaction within the LAr volume:
    # Do not count it as true pi0 -> reject this pi0
    # =================================
    if 2*res.true_n_pi0s == len(res.true_gamma_n_voxels):
        for true_pi0 in range(res.true_n_pi0s):
            if res.true_gamma_n_voxels[2*true_pi0]==0 or res.true_gamma_n_voxels[2*true_pi0+1]==0:
                print(' WARNING: reject_pi0 since true_gamma_n_voxels is 0: ', res.true_gamma_n_voxels[2*true_pi0], ' \t ', res.true_gamma_n_voxels[2*true_pi0+1])
                reject_pi0[true_pi0] = True
    else:
        print(' WARNING: true_n_pi0s =', res.true_n_pi0s, 'does not correspond to len(true_gamma_n_voxels)=', len(res.true_gamma_n_voxels))
        continue
    
    # If true pi0 decay point is OOFV:
    # Do not count it as true pi0 -> reject this pi0
    # =================================
    if 2*res.true_n_pi0s == len(res.true_gamma_n_voxels):
        for true_pi0 in range(res.true_n_pi0s):
            coord1 = np.array(res.true_gamma_pos[2*true_pi0])
            coord2 = np.array(res.true_gamma_pos[2*true_pi0+1])
            if (np.any(coord1<(0+chain.cfg['fiducialize'])) or\
                np.any(coord1>(767-chain.cfg['fiducialize'])) or\
                np.any(coord2<(0+chain.cfg['fiducialize'])) or\
                np.any(coord2>(767-chain.cfg['fiducialize'])) ):
                print(' WARNING: reject_pi0 since true_gamma_pos is not in FV: ', coord1, ' \t ', coord1)
                reject_pi0[true_pi0] = True
    else:
        print(' WARNING: true_n_pi0s =', res.true_n_pi0s, 'does not correspond to len(true_gamma_n_voxels)=', len(res.true_gamma_n_voxels))
    
    # If true_shower_first_edep is float('inf') or not in fiducial volume:
    # Do not count it as true pi0 -> reject this pi0
    # =================================
    #reject_pi0 = [False for _ in range(res.true_n_pi0s)]
    for true_pi0 in range(res.true_n_pi0s):
        #print(' res.true_shower_first_edep: ', res.true_shower_first_edep)
        if np.any(res.true_shower_first_edep[2*true_pi0]==float('inf')) or np.any(res.true_shower_first_edep[2*true_pi0+1]==float('inf')):
            print(' WARNING: reject_pi0 since true_shower_first_edep is float(inf): ', res.true_shower_first_edep[2*true_pi0], ' \t ', res.true_shower_first_edep[2*true_pi0+1])
            reject_pi0[true_pi0] = True
        coords1 = np.array(res.true_shower_first_edep[2*true_pi0])
        coords2 = np.array(res.true_shower_first_edep[2*true_pi0+1])
        if np.any(coords1<chain.cfg['fiducialize']) or np.any(coords2<chain.cfg['fiducialize']):
            print(' WARNING: reject_pi0 since true_shower_first_edep is OOFV: ', coords1, ' \t ', coords2)
            reject_pi0[true_pi0] = True
        if np.any(coords1>(767-chain.cfg['fiducialize'])) or np.any(coords2>(767-chain.cfg['fiducialize'])):
            print(' WARNING: reject_pi0 since true_shower_first_edep is OOFV: ', coords1, ' \t ', coords2)
            reject_pi0[true_pi0] = True
    
    # If true_gamma_first_step is float('inf') or not in fiducial volume:
    # Do not count it as true pi0 -> reject this pi0
    # =================================
    #reject_pi0 = [False for _ in range(res.true_n_pi0s)]
    for true_pi0 in range(res.true_n_pi0s):
        #print(' res.true_gamma_first_step: ', res.true_gamma_first_step)
        if np.any(res.true_gamma_first_step[2*true_pi0]==float('inf')) or np.any(res.true_gamma_first_step[2*true_pi0+1]==float('inf')):
            print(' WARNING: reject_pi0 since true_gamma_first_step is float(inf): ', res.true_gamma_first_step[2*true_pi0], ' \t ', res.true_gamma_first_step[2*true_pi0+1])
            reject_pi0[true_pi0] = True
        coords1 = np.array(res.true_gamma_first_step[2*true_pi0])
        coords2 = np.array(res.true_gamma_first_step[2*true_pi0+1])
        if np.any(coords1<chain.cfg['fiducialize']) or np.any(coords2<chain.cfg['fiducialize']):
            print(' WARNING: reject_pi0 since true_gamma_first_step is OOFV: ', coords1, ' \t ', coords2)
            reject_pi0[true_pi0] = True
        if np.any(coords1>(767-chain.cfg['fiducialize'])) or np.any(coords2>(767-chain.cfg['fiducialize'])):
            print(' WARNING: reject_pi0 since true_gamma_first_step is OOFV: ', coords1, ' \t ', coords2)
            reject_pi0[true_pi0] = True
    
    # If one of the true photons is depositing < 20 MeV only:
    # Do not count it as true pi0 -> reject this pi0
    # =================================
    #reject_pi0 = [False for _ in range(res.true_n_pi0s)]
    for true_pi0 in range(res.true_n_pi0s):
        if res.true_gamma_edep[2*true_pi0]<20 or res.true_gamma_edep[2*true_pi0+1]<20:
            print(' WARNING: reject_pi0 since true_gamma_edep < 20 MeV: ', res.true_gamma_edep[2*true_pi0], ' \t ',  res.true_gamma_edep[2*true_pi0+1])
            reject_pi0[true_pi0] = True
    
    res.reject_pi0 = reject_pi0
    
    print_info = False #(res.true_n_pi0s+res.reco_n_pi0s)>0 #True
    
    if print_info:
        print(' ====== Reject pi0: ', reject_pi0)
        print(' ====== Rejected pi0s: ', res.reject_pi0)
    
    
    # Print to screen:
    # =================================
    if print_info:
        #print(' -------------------------------------------------------------------------------------------------- ')
        print(' event_id:              ', res.event_id)
        print(' true_n_pi0s:           ', res.true_n_pi0s)
        print(' reco_n_pi0s:           ', res.reco_n_pi0s)
        print(' true_n_voxels:         ', res.true_gamma_n_voxels)
        print(' reject_pi0:            ', res.reject_pi0)

        #print(' true_pi0_ekin:         ', res.true_pi0_ekin)
        
        #print(' true_n_gammas:         ', res.true_n_gammas)
        #print(' true_gamma_mom:        ', res.true_gamma_mom)
        print(' true_gamma_dir:        ', res.true_gamma_dir)
        print(' true_gamma_pos:        ', res.true_gamma_pos)
        print(' true_gamma_first_step: ', res.true_gamma_first_step)
        print(' true_shower_first_edep:', res.true_shower_first_edep)
        #print(' true_gamma_ekin:       ', res.true_gamma_ekin)
        #print(' ==== true_gamma_edep:       ', res.true_gamma_edep)
        print(' true_gamma_angle:      ', res.true_gamma_angle)
        #print(' true_OOFV:             ', res.true_OOFV)

        #print(' reco_n_gammas:         ', res.reco_n_gammas)
        print(' reco_matches:          ', res.reco_matches)
        #print(' reco_gamma_mom:        ', res.reco_gamma_mom)
        print(' reco_gamma_dir:        ', res.reco_gamma_dir)
        print(' reco_gamma_start:      ', res.reco_gamma_start)
        #print(' reco_gamma_edep:       ', res.reco_gamma_edep)
        print(' reco_gamma_angle:      ', res.reco_gamma_angle)
        #print(' reco_OOFV:             ', res.reco_OOFV)
        print(' reco_pi0_mass:         ', res.reco_pi0_mass)
        #print(' -------------------------------------------------------------------------------------------------- ')


    # Obtain results for Eff & Pur:
    # =================================
    
    # Check that number of reco photons matched to a pi0 is even
    if int(res.reco_n_gammas)%2 != 0:
        print(' WARNING: odd number of reco_n_gammas!! ')
        #assert int(res.reco_n_gammas)%2 == 0
        
    # Check that number of reco_n_gammas equals reco_matches
    if int(res.reco_n_gammas) != len(res.reco_matches):
        print(' WARNING: number of reco_n_gammas != number of reco_matches!! ')
        #assert int(res.reco_n_gammas) == len(res.reco_matches)
    
    # Check that there is at least 1 true pi0:
    #if not (res.true_n_pi0s > 0):
    #    print(' WARNING: true_n_pi0s ==', res.true_n_pi0s, ' -> Cannot build a separation matrix [event: ', res.event_id, ']')
    #    assert not(res.true_n_pi0s > 0)
    
    
    # Instantiate matching_eff_pur class
    eff_pur_data = matching_eff_pur()
    
    eff_pur_data.event_id                  = res.event_id
    eff_pur_data.true_n_pi0s               = res.true_n_pi0s - sum(1 for x in reject_pi0 if x==True)
    eff_pur_data.reco_n_pi0s               = res.reco_n_pi0s
    eff_pur_data.true_pi0_ekin             = []
    eff_pur_data.true_gamma_mom            = []
    eff_pur_data.true_gamma_edep           = res.true_gamma_edep
    eff_pur_data.reco_gamma_edep           = res.reco_gamma_edep
    eff_pur_data.correctly_matched_pi0_ids = []
    eff_pur_data.wrongly_matched_pi0_ids   = []
    
    if print_info:
        print(' res.true_n_pi0s:       ', res.true_n_pi0s)
        print(' reject_pi0:            ', res.reject_pi0)
        print(' eff_pur.true_n_pi0s:   ', eff_pur_data.true_n_pi0s)
    
    for k in range(res.true_n_pi0s):
        if reject_pi0[k]:
            continue
        else:
            eff_pur_data.true_pi0_ekin.append(res.true_pi0_ekin[k])
    for k in range(res.true_n_pi0s):
        if reject_pi0[k]:
            continue
        else:
            eff_pur_data.true_gamma_mom.append(res.true_gamma_mom[2*k])
            eff_pur_data.true_gamma_mom.append(res.true_gamma_mom[2*k+1])
    eff_pur_data.reject_pi0 = res.reject_pi0

    # Define the tolerances for a reco shower to be accepted as properly matched to a true shower
    spatial_tolerance = 20 # [px]
    angular_tolerance = 20 # [degrees]
    
    # Loop over all matched photons:
#    if res.reco_n_gammas == 0:
#        print(' reco_n_gammas == 0 --> Cannot build separation matrix, set number of correctly matched pi0s to zero. ')
#    else:
    if res.reco_n_gammas != 0:
        # Build a separation matrix and angle matrix
        sep_mat = np.full((int(res.true_n_gammas), int(res.reco_n_gammas)), float('inf')) # true_gammas vertically, reco_gammas horizontally
        ang_mat = np.full((int(res.true_n_gammas), int(res.reco_n_gammas)), float('inf')) # true_gammas vertically, reco_gammas horizontally
        
        if print_info:
            #print(' -------------------------------------------------------------------------------------------------- ')
            print(' ================ EVENT PASSED SELECTION: ================ ')
            print(' event_id:               ', res.event_id)
            print(' true_n_pi0s:            ', res.true_n_pi0s)
            print(' reco_n_pi0s:            ', res.reco_n_pi0s)
            print(' reco_n_gammas:          ', res.reco_n_gammas)
            print(' true_gamma_first_step:  ', res.true_gamma_first_step)
            print(' true_shower_first_edep: ', res.true_shower_first_edep)
            print(' reco_gamma_start:       ', res.reco_gamma_start)
            print(' true_gamma_dir:         ', res.true_gamma_dir)
            print(' reco_gamma_dir:         ', res.reco_gamma_dir)
        
        for i in range(int(res.true_n_gammas)):
            for j in range(int(res.reco_n_gammas)):
#                print(' i: ', i, ' \t j: ', j, ' \t ', (res.reco_gamma_start[j]-res.true_gamma_pos[i]))
#                print(' res.true_gamma_first_step', np.array(res.true_gamma_first_step[i]))
#                print(' res.reco_gamma_start', np.array(res.reco_gamma_start[j]))
                #distance = np.linalg.norm(np.array(res.true_gamma_first_step[i])-np.array(res.reco_gamma_start[j]))
                distance = np.linalg.norm(np.array(res.true_shower_first_edep[i])-np.array(res.reco_gamma_start[j]))
                angle    = np.arccos(np.dot(res.reco_gamma_dir[j],res.true_gamma_dir[i]))*360./(2.*np.pi)
                #print(' distance [px]: ', distance)
                #print(' angle [deg]:   ', angle)
                #if distance < spatial_tolerance:
                #    sep_mat[i,j] = distance
                sep_mat[i,j] = distance
                ang_mat[i,j] = angle
                #else:
                #    sep_mat[i,j] = float('inf')
                #if angle < angular_tolerance:
                #    ang_mat[i,j] = angle
                #else:
                #    ang_mat[i,j] = float('inf')
        #print(' true_gamma_first_step: ', res.true_gamma_first_step)
        #print(' true_shower_first_edep: ', res.true_shower_first_edep)
        #print(' reco_gamma_start: ', res.reco_gamma_start)
#        print(' sep_mat: \n', sep_mat)
#        print(' ang_mat: \n', ang_mat)
        
        for i in range(int(res.true_n_gammas)):
            for j in range(int(res.reco_n_gammas)):
                if (sep_mat[i][j] > spatial_tolerance) or (ang_mat[i][j] > angular_tolerance):
                    sep_mat[i][j] = float('inf')
                    ang_mat[i][j] = float('inf')
#        print(' sep_mat new: \n', sep_mat)
#        print(' ang_mat new: \n', ang_mat)
        
        
        # Assign reco_gamma to true_gamma (taking sequencially the smallest index from the sep_mat)
        assignments = [] # list of U arrays (where U is min(n_clusters,n_true_gammas)). First array entry: cluster index, second array: true_gamma index.
        for i in range(min(sep_mat.shape)):
            if sep_mat.min() < spatial_tolerance:
                indices = np.argwhere(sep_mat.min() == sep_mat)
                assignments.append(indices[0])
                sep_mat[indices[0][0],:] = float('inf')
                sep_mat[:,indices[0][1]] = float('inf')
#            else:
#                print(' sep_mat.min() =', sep_mat.min(), '>', spatial_tolerance, '= spatial_tolerance -> No action')
#        print(' assignments: ', assignments)
#        print(' sep_mat:\n', sep_mat)
#        print(' ang_mat:\n', ang_mat)
#        print(' len(assignments): ', len(assignments))
#        if len(assignments) < 2:
#            print(' eff_pur_data.correctly_matched_n_pi0 should be set to 0 ')
#        if len(assignments) > 2:
#            print(' WARNING: len(assignments) =', len(assignments), ' (assignments =', assignments, '[event: ', res.event_id, ']')


        # Check if the assignments should be rejected by the angle between true_dir and reco_dir
        reject_due_to_angle = []
        for i, ass in enumerate(assignments):
            if ang_mat[ass[0]][ass[1]] > angular_tolerance:
                reject_due_to_angle.append(i)

        # Check if assignments correspond (pair-wise) to the correct true_pi0
        # if assignment = (x,y) = (even-even), then (x+1, y+1) should be found in assignments in order to be properly matched
        # if assignment = (x,y) = (even-odd),  then (x+1, y-1) should be found in assignments in order to be properly matched
        # if assignment = (x,y) = (odd-even),  then (x-1, y+1) should be found in assignments in order to be properly matched
        # if assignment = (x,y) = (odd-odd),   then (x-1, y-1) should be found in assignments in order to be properly matched
        counter_correctly_matched_pi0 = 0
        correctly_matched_pi0_indices = []
        counter_wrongly_matched_pi0   = 0
        wrongly_matched_pi0_indices   = []
        for i, ass in enumerate(assignments):
            proper_match = False
            #print(' ass: ', ass)
            if ass[0]%2 == 0 and ass[1]%2 == 0:
                #print(' even-even')
                for j, ass_test in enumerate(assignments):
                    #print(' ass_test: ', ass_test)
                    if (ass_test[0]==ass[0]+1) and (ass_test[1]==ass[1]+1):
                        #print(' proper match: ', ass_test)
                        proper_match = True
                        if ass[0]%2==0 and ass[0] not in correctly_matched_pi0_indices:
                            if i not in reject_due_to_angle and j not in reject_due_to_angle:
                                correctly_matched_pi0_indices.append(int(ass[0]/2))
                            else:
                                _ang = np.arccos(np.dot(res.reco_gamma_dir[ass[1]],res.true_gamma_dir[ass[0]]))*360./(2.*np.pi)
                                print(' Rejected assignment', ass, 'due to angle (',_ang, 'degrees)...')
                        break
                    #else:
                    #    print(' no match...')
            elif ass[0]%2 == 0 and ass[1]%2 == 1:
                #print(' even-odd')
                for j, ass_test in enumerate(assignments):
                    #print(' ass_test: ', ass_test)
                    if (ass_test[0]==ass[0]+1) and (ass_test[1]==ass[1]-1):
                        #print(' proper match: ', ass_test)
                        proper_match = True
                        if ass[0]%2==0 and ass[0] not in correctly_matched_pi0_indices:
                            if i not in reject_due_to_angle and j not in reject_due_to_angle:
                                correctly_matched_pi0_indices.append(int(ass[0]/2))
                            else:
                                _ang = np.arccos(np.dot(res.reco_gamma_dir[ass[1]],res.true_gamma_dir[ass[0]]))*360./(2.*np.pi)
                                print(' Rejected assignment', ass, 'due to angle (',_ang, 'degrees)...')
                        break
                    #else:
                    #    print(' no match...')
            elif ass[0]%2 == 1 and ass[1]%2 == 0:
                #print(' odd-even')
                for j, ass_test in enumerate(assignments):
                    #print(' ass_test: ', ass_test)
                    if (ass_test[0]==ass[0]-1) and (ass_test[1]==ass[1]+1):
                        #print(' proper match: ', ass_test)
                        proper_match = True
                        if ass[0]%2==0 and ass[0] not in correctly_matched_pi0_indices:
                            if i not in reject_due_to_angle and j not in reject_due_to_angle:
                                correctly_matched_pi0_indices.append(int(ass[0]/2))
                            else:
                                _ang = np.arccos(np.dot(res.reco_gamma_dir[ass[1]],res.true_gamma_dir[ass[0]]))*360./(2.*np.pi)
                                print(' Rejected assignment', ass, 'due to angle (',_ang, 'degrees)...')
                        break
                    #else:
                    #    print(' no match...')
            elif ass[0]%2 == 1 and ass[1]%2 == 1:
                #print(' odd-odd')
                for j, ass_test in enumerate(assignments):
                    #print(' ass_test: ', ass_test)
                    if (ass_test[0]==ass[0]-1) and (ass_test[1]==ass[1]-1):
                        #print(' proper match: ', ass_test)
                        proper_match = True
                        if ass[0]%2==0 and ass[0] not in correctly_matched_pi0_indices:
                            if i not in reject_due_to_angle and j not in reject_due_to_angle:
                                correctly_matched_pi0_indices.append(int(ass[0]/2))
                            else:
                                _ang = np.arccos(np.dot(res.reco_gamma_dir[ass[1]],res.true_gamma_dir[ass[0]]))*360./(2.*np.pi)
                                print(' Rejected assignment', ass, 'due to angle (',_ang, 'degrees)...')
                        break
            else:
                print(' WARNING: assignment', ass, 'not recognized!! [event: ', res.event_id, ']')
            
            if proper_match:
                counter_correctly_matched_pi0 += 1
#                print(' proper_match: ', proper_match)
#            if not proper_match:
#                print(' No proper match found ')
#        print(' correctly_matched_pi0_indices: ', correctly_matched_pi0_indices)
        for index in range(res.reco_n_pi0s):
            if index not in correctly_matched_pi0_indices:
                wrongly_matched_pi0_indices.append(index)
        #print(' wrongly_matched_pi0_indices:   ', wrongly_matched_pi0_indices)
        #print(' total matched:                 ', res.reco_n_pi0s)

        if (counter_correctly_matched_pi0%2 != 0):
            print(' ERROR: counter for correctly matched pairs must be an even number, but it is ', counter_correctly_matched_pi0, ' [event: ', res.event_id, ']')
            #assert (counter_correctly_matched_pi0%2 == 0)
        else:
            eff_pur_data.correctly_matched_n_pi0 = int(counter_correctly_matched_pi0/2)
        
        if len(correctly_matched_pi0_indices) > 0:
            eff_pur_data.correctly_matched_pi0_ids = correctly_matched_pi0_indices# = np.array(correctly_matched_pi0_indices)
        if len(wrongly_matched_pi0_indices) > 0:
            eff_pur_data.wrongly_matched_pi0_ids = wrongly_matched_pi0_indices# = np.array(wrongly_matched_pi0_indices)
        #else:
        #    eff_pur_data.correctly_matched_pi0_ids = []
        #print(eff_pur_data.correctly_matched_pi0_ids)
        #print(' ---------------------------------- ')

#    print(' -------------------------------------------------------------------------------------------------- ')


    # Append the extracted data to the ResultsList
    #print(eff_pur_data.event_id, eff_pur_data.true_n_pi0s, eff_pur_data.correctly_matched_n_pi0, eff_pur_data.reco_n_pi0s)
    eff_pur_list.append(eff_pur_data)

    
    # Update progress bar
    out.update(progress(res.event_id,len(ResultsList),'images'))
out.update(progress(len(ResultsList),len(ResultsList),'images'))
'''

print(' Done ')

In [ ]:
import pandas as pd
import seaborn
from matplotlib import pyplot as plt

seaborn.set(rc={'figure.figsize':(15, 10),})
seaborn.set_context('talk') # or paper


# Define histogram range and binning
x_min    = 0
x_max    = 500
n_bins_x = 10
x_bins = np.linspace(x_min,x_max,n_bins_x+1)

correctly_matched   = [ 0 for _ in range(n_bins_x)]
wrongly_matched     = [ 0 for _ in range(n_bins_x)]
total_true_pi0      = [ 0 for _ in range(n_bins_x)]
total_reco_pi0      = [ 0 for _ in range(n_bins_x)]
matching_eff        = [-9 for _ in range(n_bins_x)]
matching_eff_uncert = [ 0 for _ in range(n_bins_x)]
matching_pur        = [-9 for _ in range(n_bins_x)]
matching_pur_uncert = [ 0 for _ in range(n_bins_x)]

event_ids_correctly_matched    = []
event_ids_wrongly_matched      = []
event_ids_no_reconstructed_pi0 = []

# Count pi0s for statistics
for event in eff_pur_list:

    #print(' -------------------------------------------------------------------------------------------------------- ')
    #print(' event_id:        ', event.event_id)
    #print(' n true pi0s:     ', event.true_n_pi0s)
    #print(' n reco pi0s:     ', event.reco_n_pi0s)
    #print(' reject_pi0:      ', event.reject_pi0)
    #print(' true_gamma_edep: ', event.true_gamma_edep)
    
    if event.reco_n_pi0s > event.true_n_pi0s:
        print(' WARNING: n_reco_pi0s =', event.reco_n_pi0s, ' > ', event.true_n_pi0s, '= n_true_pi0s [event: ', event.event_id, ']')
    
    if len(event.true_pi0_ekin) > 0: # and event.reco_n_pi0s>0:
        # Loop over all true pi0s
        for true_pi0 in range(event.true_n_pi0s):
            if event.true_pi0_ekin[true_pi0]>x_max:
                continue
            #print(' --------------------------- ')
            #print(' event_id:    ', event.event_id)
            #print(' true_n_pi0s: ', event.true_n_pi0s)
            #print(' reco_n_pi0s: ', event.reco_n_pi0s)
            #print(' true_pi0_ekin: ', event.true_pi0_ekin)
            if true_pi0 in event.correctly_matched_pi0_ids:
                #print(' Correctly matched [event:', event.event_id,']')
                assigned_to_bin = False
                for bin_index in range(n_bins_x):
                    if event.true_pi0_ekin[true_pi0] > x_bins[bin_index] and event.true_pi0_ekin[true_pi0] < x_bins[bin_index+1]:
                        assigned_to_bin = True
                        correctly_matched[bin_index] += 1 #event.correctly_matched_n_pi0
                        total_true_pi0[bin_index]    += 1 #event.true_n_pi0s
                        total_reco_pi0[bin_index]    += 1 #event.reco_n_pi0s
                        event_ids_correctly_matched.append(event.event_id)
                        #print(' pi0 with true ekin', event.true_pi0_ekin[true_pi0], 'has been assigned to bin', bin_index, 'with values between', x_bins[bin_index], 'and', x_bins[bin_index+1])
                if not assigned_to_bin:
                    print(' WARNING: pi0 with true ekin', event.true_pi0_ekin[true_pi0], 'has not been assigned to a bin! ')
            
            elif true_pi0 in event.wrongly_matched_pi0_ids:
                #print(' Wrongly matched [event:', event.event_id,']')
                assigned_to_bin = False
                for bin_index in range(n_bins_x):
                    if event.true_pi0_ekin[true_pi0] > x_bins[bin_index] and event.true_pi0_ekin[true_pi0] < x_bins[bin_index+1]:
                        assigned_to_bin = True
                        wrongly_matched[bin_index]   += 1
                        total_true_pi0[bin_index]    += 1 #event.true_n_pi0s
                        total_reco_pi0[bin_index]    += 1 #event.reco_n_pi0s
                        event_ids_wrongly_matched.append(event.event_id)
                        #print(' pi0 with true ekin', event.true_pi0_ekin[true_pi0], 'has been assigned to bin', bin_index, 'with values between', x_bins[bin_index], 'and', x_bins[bin_index+1])
                if not assigned_to_bin:
                    print(' WARNING: pi0 with true ekin', event.true_pi0_ekin[true_pi0], 'has not been assigned to a bin! ')
            else:
                #print(' NOTE: true_pi0 index =', true_pi0, 'neither in correctly_matched nor in wrongly_matched list... ')
                assigned_to_bin = False
                for bin_index in range(n_bins_x):
                    if event.true_pi0_ekin[true_pi0] > x_bins[bin_index] and event.true_pi0_ekin[true_pi0] < x_bins[bin_index+1]:
                        assigned_to_bin = True
                        total_true_pi0[bin_index]    += 1 #event.true_n_pi0s
                        event_ids_no_reconstructed_pi0.append(event.event_id)
                        #print(' pi0 with true ekin', event.true_pi0_ekin[true_pi0], 'has been assigned to bin', bin_index, 'with values between', x_bins[bin_index], 'and', x_bins[bin_index+1])
                if not assigned_to_bin:
                    print(' WARNING: pi0 with true ekin', event.true_pi0_ekin[true_pi0], 'has not been assigned to a bin! ')
        
    '''
        for true_pi0 in range(event.true_n_pi0s):
            
            # Correctly matched
            if true_pi0 in event.correctly_matched_pi0_ids:
#                print('\n', true_pi0, 'in correctly_matched_pi0_ids =', event.correctly_matched_pi0_ids, '--> CORRECTLY matched! ')
                assigned_to_bin = False
                for bin_index in range(n_bins_x):
                    if event.true_pi0_ekin[true_pi0] > x_bins[bin_index] and event.true_pi0_ekin[true_pi0] < x_bins[bin_index+1]:
                        assigned_to_bin = True
                        correctly_matched[bin_index] += 1 #event.correctly_matched_n_pi0
                        total_true_pi0[bin_index]    += 1 #event.true_n_pi0s
                        total_reco_pi0[bin_index]    += 1 #event.reco_n_pi0s
                        event_ids_correctly_matched.append(event.event_id)
#                        print(' pi0 with true ekin', event.true_pi0_ekin[true_pi0], 'has been assigned to bin', bin_index, 'with values between', x_bins[bin_index], 'and', x_bins[bin_index+1])
                if not assigned_to_bin:
                    print(' WARNING: pi0 with true ekin', event.true_pi0_ekin[true_pi0], 'has not been assigned to a bin! ')

            # Not correctly matched
            else:
                print(' true_pi0: ', true_pi0, ' \t wrongly_matched_ids: ', event.wrongly_matched_pi0_ids)
#                print('\n', true_pi0, ' NOT in correctly_matched_pi0_ids =', event.correctly_matched_pi0_ids, '--> NOT matched! ')
                assigned_to_bin = False
                for bin_index in range(n_bins_x):
                    if event.true_pi0_ekin[true_pi0] > x_bins[bin_index] and event.true_pi0_ekin[true_pi0] < x_bins[bin_index+1]:
                        assigned_to_bin = True
                        total_true_pi0[bin_index]    += 1 #event.true_n_pi0s
                        #total_reco_pi0[bin_index]    += 1 #event.reco_n_pi0s
                        event_ids_wrongly_matched.append(event.event_id)
#                        print(' pi0 with true ekin', event.true_pi0_ekin[true_pi0], 'has been assigned to bin', bin_index, 'with values between', x_bins[bin_index], 'and', x_bins[bin_index+1])
                if not assigned_to_bin:
                    print(' WARNING: pi0 with true ekin', event.true_pi0_ekin[true_pi0], 'has not been assigned to a bin! ')  
        
    else:
        print(' WARNING: event.true_pi0_ekin does not exist! [event: ', event.event_id, ']')
    '''
    
#    print(' correctly_matched: ', correctly_matched, '\t entries: ', np.sum(correctly_matched))
#    print(' total_true_pi0:    ', total_true_pi0,    '\t entries: ', np.sum(total_true_pi0))
#    print(' total_reco_pi0:    ', total_reco_pi0,    '\t entries: ', np.sum(total_reco_pi0))
#    print(' //////////////////////////////////////////////////////////////////////////////////////////////////////////////// ')


print(' correctly_matched: (',np.sum(correctly_matched), ') \t ', correctly_matched)
print(' wrongly_matched:   (',np.sum(wrongly_matched),   ') \t ', wrongly_matched)
print(' total_true_pi0:    (',np.sum(total_true_pi0),    ') \t ', total_true_pi0)
print(' total_reco_pi0:    (',np.sum(total_reco_pi0),    ') \t ', total_reco_pi0)
print(' event_ids_correctly_matched =    ', event_ids_correctly_matched)
print(' event_ids_wrongly_matched =      ', event_ids_wrongly_matched)
print(' event_ids_no_reconstructed_pi0 = ', event_ids_no_reconstructed_pi0)


# Obtain efficiencies and purities
x_list     = []
x_err_list = []
for i in range(n_bins_x):
    x_list.append((x_bins[i]+x_bins[i+1])/2.)
    x_err_list.append((x_bins[i+1]-x_bins[i])/2.)
    if total_true_pi0[i]>0:
        matching_eff[i]       = correctly_matched[i] / total_true_pi0[i]
        matching_eff_uncert[i] = np.sqrt( (correctly_matched[i]*(1.-correctly_matched[i]/total_true_pi0[i]))/(total_true_pi0[i]**2) )
        #matching_eff[i]       = total_reco_pi0[i] / total_true_pi0[i]
        #matching_eff_uncert[i] = np.sqrt( (total_reco_pi0[i]*(1.-total_reco_pi0[i]/total_true_pi0[i]))/(total_true_pi0[i]**2) )
    if total_reco_pi0[i]>0:
        matching_pur[i]       = correctly_matched[i] / total_reco_pi0[i]
        matching_pur_uncert[i] = np.sqrt( (correctly_matched[i]*(1.-correctly_matched[i]/total_reco_pi0[i]))/(total_reco_pi0[i]**2) )
        #matching_pur[i]       = correctly_matched[i] / total_reco_pi0[i]
        #matching_pur_uncert[i] = np.sqrt( (correctly_matched[i]*(1.-correctly_matched[i]/total_reco_pi0[i]))/(total_reco_pi0[i]**2) )
        
overall_eff = np.sum(correctly_matched) / np.sum(total_true_pi0)
overall_pur = np.sum(correctly_matched) / np.sum(total_reco_pi0)

'''
# Obtain efficiencies and purities
x_list     = []
x_err_list = []
for i in range(n_bins_x):
    x_list.append((x_bins[i]+x_bins[i+1])/2.)
    x_err_list.append((x_bins[i+1]-x_bins[i])/2.)
    if total_true_pi0[i]>0:
        matching_eff[i]        = correctly_matched[i] / total_true_pi0[i]
        matching_eff_uncert[i] = np.sqrt( (correctly_matched[i]*(1.-correctly_matched[i]/total_true_pi0[i]))/(total_true_pi0[i]**2) )
    if total_reco_pi0[i]>0:
        matching_pur[i]        = correctly_matched[i] / total_reco_pi0[i]
        matching_pur_uncert[i] = np.sqrt( (correctly_matched[i]*(1.-correctly_matched[i]/total_reco_pi0[i]))/(total_reco_pi0[i]**2) )
'''

print(' x_bins:              ', x_bins)
print(' x_list:              ', x_list)
print(' x_err_list:          ', x_err_list)
print(' matching_eff:        ', matching_eff)
print(' matching_eff_uncert: ', matching_eff_uncert)
print(' matching_pur:        ', matching_pur)
print(' matching_pur_uncert: ', matching_pur_uncert)


# Define parameters of the frame
fig = plt.figure() # plt.figure(figsize=(width,height))
#fig.patch.set_facecolor('white')
#fig.patch.set_alpha(0.0)
ax = fig.add_subplot(111)
#ax.patch.set_facecolor('#ababab') # #ababab
ax.patch.set_alpha(0.0)
ax.spines['bottom'].set_color('0.5') #'black', ...
ax.spines['bottom'].set_linewidth(2)
ax.spines['bottom'].set_visible(True)
ax.spines['top'].set_color('0.5')
ax.spines['top'].set_linewidth(2)
ax.spines['top'].set_visible(True)
ax.spines['right'].set_color('0.5')
ax.spines['right'].set_linewidth(2)
ax.spines['right'].set_visible(True)
ax.spines['left'].set_color('0.5')
ax.spines['left'].set_linewidth(2)
ax.spines['left'].set_visible(True)


# Ticks, grid and ticks labels
ax.tick_params(direction='in', length=10, width=2,                  # direction, length and width of the ticks (in, out, inout)
                colors='0.5',                                       # color of the ticks ('black', '0.5')
                bottom=True, top=True, right=True, left=True,       # whether to draw the respective ticks
                zorder = 10.,                                       # tick and label zorder
                pad = 10.,                                          # distance between ticks and tick labels
                labelsize = 17,                                     # size of the tick labels
                labelright=False, labeltop=False)                   # wether to draw the tick labels on axes
                #labelrotation=45.                                  # rotation of the labels
                #grid_color='black',                                # grid
                #grid_alpha=0.0,
                #grid_linewidth=1.0,
# colors='black','0.5'


# Plot
plt.errorbar(x_list, matching_eff, xerr=x_err_list, yerr=matching_eff_uncert, label='Efficiency (overall: %.2f)' %overall_eff, fmt='o', color='r', alpha=0.8)
plt.errorbar(x_list, matching_pur, xerr=x_err_list, yerr=matching_pur_uncert, label='Purity (overall: %.2f)' %overall_pur, fmt='x', color='g', alpha=0.8)
#n, bins, patches = plt.hist(matching_eff, bins=n_bins_x, range=[x_min,x_max], histtype='step', stacked=False, linewidth=3, alpha=0.8, color='red')

# Axis labels
plt.xlabel('True $\pi^0$ ekin [MeV]', fontsize=20, labelpad=20)
plt.ylabel('Matching Efficiency & Purity', fontsize=20, labelpad=20)

plt.ylim(bottom=-0.1, top=1.3)
#plt.yscale('log') # linear, log
#plt.yscale('symlog', linthreshy=1)

#plt.axvline(134.9770, color='r', linestyle='dashed', linewidth=2, label='Literature value for $\pi^0$ mass')
#plt.axhline(overall_eff, color='r', linestyle='dashed', linewidth=2, label='Overall efficiency (%.2f)' %overall_eff)
#plt.axhline(overall_pur, color='g', linestyle='dashed', linewidth=2, label='Overall purity (%.2f)' %overall_pur)

# Legend
#entry_0 = 'Efficiency'
#entry_1 = 'Purity'
plt.legend(loc=[0.65,0.05], prop={'size': 17}) # loc='upper right'


# Save figure
#fig_name = '0' + folder + '_pi0_mass_peak.png'

# Get file name
#print(io_cfg)
start_index = io_cfg.find('data_keys:') + 10
end_index = io_cfg.find('limit_num_files:')
words = io_cfg[start_index:end_index].split('.root')
for i in range(len(words)):
    #print(words[i])
    if words[i].find('#')>0:
        continue
    else:
        start_index = words[i].find('data/')
        file_name = words[i][start_index+5:]
        break
#print(' filename = ', file_name)

fig_name = 'matching_eff_pur_' +\
            str(data_size) + 'events_' +\
            'from_file_' + file_name + '_root.png'
#            'from_file_' + str(chain.io_cfg['iotool']['dataset']['data_keys'])[13:-7] +'_root.png'
#          ' SELECTION_CUTS:
print(' fig_name: ', fig_name)
if True:
    plt.savefig('plots/' + fig_name, dpi=400) # bbox_inches='tight'

In [ ]:
print(' correctly_matched: (',np.sum(correctly_matched), ') \t ', correctly_matched)
print(' wrongly_matched:   (',np.sum(wrongly_matched),   ') \t ', wrongly_matched)
print(' total_true_pi0:    (',np.sum(total_true_pi0),    ') \t ', total_true_pi0)
print(' total_reco_pi0:    (',np.sum(total_reco_pi0),    ') \t ', total_reco_pi0)

In [ ]:
event_ids_correctly_matched    = []
event_ids_wrongly_matched      = []
event_ids_no_reconstructed_pi0 = []
\
not_matched_list_smaller_true_gamma_edep = []
not_matched_list_larger_true_gamma_edep  = []
not_matched_list_smaller_reco_gamma_edep = []
not_matched_list_larger_reco_gamma_edep  = []

correctly_matched_list_smaller_true_gamma_edep = []
correctly_matched_list_larger_true_gamma_edep  = []
correctly_matched_list_smaller_reco_gamma_edep = []
correctly_matched_list_larger_reco_gamma_edep  = []

for event in eff_pur_list:
    
    if event.event_id in event_ids_wrongly_matched:
        if event.true_n_pi0s==1: # TODO: CHANGE IT FOR EVENTS WITH >1 PI0
            not_matched_list_smaller_true_gamma_edep.append(np.amin(event.true_gamma_edep))
            not_matched_list_larger_true_gamma_edep.append(np.amax(event.true_gamma_edep))
        if event.reco_n_pi0s==1: # TODO: CHANGE IT FOR EVENTS WITH >1 PI0
            not_matched_list_smaller_reco_gamma_edep.append(np.amin(event.reco_gamma_edep))
            not_matched_list_larger_reco_gamma_edep.append(np.amax(event.reco_gamma_edep))
            
    if event.event_id in event_ids_correctly_matched:
        if event.true_n_pi0s==1: # TODO: CHANGE IT FOR EVENTS WITH >1 PI0
            correctly_matched_list_smaller_true_gamma_edep.append(np.amin(event.true_gamma_edep))
            correctly_matched_list_larger_true_gamma_edep.append(np.amax(event.true_gamma_edep))
        if event.reco_n_pi0s==1: # TODO: CHANGE IT FOR EVENTS WITH >1 PI0
            correctly_matched_list_smaller_reco_gamma_edep.append(np.amin(event.reco_gamma_edep))
            correctly_matched_list_larger_reco_gamma_edep.append(np.amax(event.reco_gamma_edep))
        
        '''
        print(' ----------------------------------------------------------- ')
        print(' event_id:                  ', event.event_id)
        print(' true_n_pi0s:               ', event.true_n_pi0s)
        print(' reco_n_pi0s:               ', event.reco_n_pi0s)
        print(' true_pi0_ekin:             ', event.true_pi0_ekin)
        print(' true_gamma_mom:            ', event.true_gamma_mom)
        print(' true_gamma_edep:           ', event.true_gamma_edep)
        #print(' correctly_matched_n_pi0:   ', event.correctly_matched_n_pi0)
        print(' correctly_matched_pi0_ids: ', event.correctly_matched_pi0_ids)
        print(' wrongly_matched_pi0_ids:   ', event.wrongly_matched_pi0_ids)
        print(' reject_pi0:                ', event.reject_pi0)
        '''
        
#print(' \n not_matched_list_smaller_true_gamma_edep: ', not_matched_list_smaller_true_gamma_edep)
#print(' \n not_matched_list_larger_true_gamma_edep:  ', not_matched_list_larger_true_gamma_edep)
#print(' \n not_matched_list_smaller_reco_gamma_edep: ', not_matched_list_smaller_reco_gamma_edep)
#print(' \n not_matched_list_larger_reco_gamma_edep:  ', not_matched_list_larger_reco_gamma_edep)

print('Done')

In [ ]:
import pandas as pd
import seaborn
from matplotlib import pyplot as plt

seaborn.set(rc={'figure.figsize':(15, 10),})
seaborn.set_context('talk') # or paper

# Define histogram range and binning
x_min    = 0
x_max    = 500
n_bins_x = 50
x_bins = np.linspace(x_min,x_max,n_bins_x+1)

# Define parameters of the frame
fig = plt.figure() # plt.figure(figsize=(width,height))
#fig.patch.set_facecolor('white')
#fig.patch.set_alpha(0.0)
ax = fig.add_subplot(111)
#ax.patch.set_facecolor('#ababab') # #ababab
ax.patch.set_alpha(0.0)
ax.spines['bottom'].set_color('0.5') #'black', ...
ax.spines['bottom'].set_linewidth(2)
ax.spines['bottom'].set_visible(True)
ax.spines['top'].set_color('0.5')
ax.spines['top'].set_linewidth(2)
ax.spines['top'].set_visible(True)
ax.spines['right'].set_color('0.5')
ax.spines['right'].set_linewidth(2)
ax.spines['right'].set_visible(True)
ax.spines['left'].set_color('0.5')
ax.spines['left'].set_linewidth(2)
ax.spines['left'].set_visible(True)

# Ticks, grid and ticks labels
ax.tick_params(direction='in', length=10, width=2,                  # direction, length and width of the ticks (in, out, inout)
                colors='0.5',                                       # color of the ticks ('black', '0.5')
                bottom=True, top=True, right=True, left=True,       # whether to draw the respective ticks
                zorder = 10.,                                       # tick and label zorder
                pad = 10.,                                          # distance between ticks and tick labels
                labelsize = 17,                                     # size of the tick labels
                labelright=False, labeltop=False)                   # wether to draw the tick labels on axes
                #labelrotation=45.                                  # rotation of the labels
                #grid_color='black',                                # grid
                #grid_alpha=0.0,
                #grid_linewidth=1.0,
# colors='black','0.5'

# Plot n_edeps
#n0, bins0, patches0 = plt.hist([not_matched_list_smaller_true_gamma_edep], label='not matched smaller true',  bins=n_bins_x, range=[x_min,x_max], histtype='stepfilled', stacked=False, linewidth=3, alpha=0.5)
#n1, bins1, patches1 = plt.hist([not_matched_list_larger_true_gamma_edep],  label='not matched larger true',   bins=n_bins_x, range=[x_min,x_max], histtype='stepfilled', stacked=False, linewidth=3, alpha=0.5)
n2, bins2, patches2 = plt.hist([not_matched_list_smaller_reco_gamma_edep], label='not matched smaller reco',  bins=n_bins_x, range=[x_min,x_max], histtype='stepfilled', stacked=False, linewidth=5, alpha=0.5)
n3, bins3, patches3 = plt.hist([not_matched_list_larger_reco_gamma_edep],  label='not matched larger reco',   bins=n_bins_x, range=[x_min,x_max], histtype='stepfilled', stacked=False, linewidth=5, alpha=0.5)

#n4, bins4, patches4 = plt.hist([correctly_matched_list_smaller_true_gamma_edep], label='correctly matched smaller true',  bins=n_bins_x, range=[x_min,x_max], histtype='stepfilled', stacked=False, linewidth=3, alpha=0.5)
#n5, bins5, patches5 = plt.hist([correctly_matched_list_larger_true_gamma_edep],  label='correctly matched larger true',   bins=n_bins_x, range=[x_min,x_max], histtype='stepfilled', stacked=False, linewidth=3, alpha=0.5)
n6, bins6, patches6 = plt.hist([correctly_matched_list_smaller_reco_gamma_edep], label='correctly matched smaller reco',  bins=n_bins_x, range=[x_min,x_max], histtype='stepfilled', stacked=False, linewidth=5, alpha=0.5)
n7, bins7, patches7 = plt.hist([correctly_matched_list_larger_reco_gamma_edep],  label='correctly matched larger reco',   bins=n_bins_x, range=[x_min,x_max], histtype='stepfilled', stacked=False, linewidth=5, alpha=0.5)

# Legend
plt.legend(loc=[0.55,0.7], prop={'size': 17}) # loc='upper right'

# Axis labels
plt.xlabel('True & reco shower deposited energy [MeV]', fontsize=20, labelpad=20)
plt.ylabel('Entries [-]', fontsize=20, labelpad=20)

#plt.ylim(bottom=0, top=200)
#plt.yscale('log') # linear, log
#plt.yscale('symlog', linthreshy=1)
#plt.axvline(134.9770, color='r', linestyle='dashed', linewidth=2, label='Literature value for $\pi^0$ mass')

# Save figure
#fig_name = '0' + folder + '_pi0_mass_peak.png'
#fig_name = 'pi0_mass_02_' +\
#            str(data_size) + 'ev_' +\
#            'refit_dir_' + str(chain.cfg['refit_dir']) + '_' +\
#            'fiducial_0px_' +\
#            'no_selection_cut.png'
#plt.savefig('plots/' + fig_name, dpi=400) # bbox_inches='tight'